<a href="https://colab.research.google.com/github/ben0it8/poetry-language-model/blob/master/jozsef_language_model_wordpiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Check GPU memory availability

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 249.5 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


##### Install extra requirements

In [94]:
!pip install -U ftfy
!pip install -U sentencepiece

Requirement already up-to-date: ftfy in /usr/local/lib/python3.6/dist-packages (5.5.1)
Requirement already up-to-date: sentencepiece in /usr/local/lib/python3.6/dist-packages (0.1.81)


In [0]:
import os, sys
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%matplotlib inline
%config InlineBackend.figure_format='retina'; # adapt plots for retina displays
import seaborn as sns
sns.set_style('darkgrid');
sns.set_context(context='notebook');
from typing import List
from tqdm import tqdm_notebook as tqdm
from pathlib import Path
from pprint import pprint
import numpy as np 
import math
import time
import pandas as pd
import io
import ftfy 
import re
import requests
import os
import torch
from collections import defaultdict
import dill
import pickle

import torch
from torch import nn, optim, tensor
from torch.nn import functional as F
from random import sample

from google.colab import files
from zipfile import ZipFile

import sentencepiece as spm

logger = logging.getLogger(__name__)

data_dir = Path("data/jozsef-wp").resolve()
data_dir.mkdir(exist_ok=True, parents=True)
url = "https://raw.githubusercontent.com/ben0it8/ady/master/data/jozsef.txt" 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
def get_file_paths(path):
  path = Path(path)
  ret = []
  for x in path.glob('*'):
    if x.is_file() and not str(x).endswith('.zip'):
      ret.append(str(x))
  return ret

def zip_dir(path, name):
  if not name.endswith('.zip'): name += '.zip'
  file_paths = get_file_paths(path)
  
  with ZipFile(path/name, 'w') as zip:
    for file in file_paths:
      zip.write(file, os.path.basename(file))
  print(f"Zipped files at {path}")
  return (path/name).resolve()

##Read data

## Clean data & write to disk

In [97]:
def is_title(t):
  t = re.sub('[\d\s\s\-!?,]', '', t)
  return t.isupper()

def is_dateline(t):
  if any(m in t for m in ['jan.', 'febr.', 'márc.', 'ápr.', 'jún.', 'júl.', 'aug.', 'szept.',
      'okt.', 'nov.', 'dec.']):
    return True
  else: False
    
def fix_text(t:str):
  t = ftfy.fix_text(t, normalization='NFC')
  t = t.replace('\n', '') # remove newlines
  t = re.sub(r'[»«]', '', t) # remove special parenthesis
  t = re.sub(r'[0-9]','', t)
  t = re.sub("\s\s+", " ", t) # skip whitespaces
  t = t.strip()
  return t
    
def fix_texts(texts:list):
  out = []
  for i, line in enumerate(texts):
    if "\u2424" in line:
      line = line.split("\u2424")
    elif '\u2028' in line:
      line = line.split('\u2028')
    elif '\u000A' in line:
      line = line.split('\u000A')
    else:
      line = [line] 
    for t in line:
      t = fix_text(t)
      if (t is None or len(t.replace(' ', ''))<=3 or is_title(t) or
          t.startswith(('.', ',', '?', '!', '-', ';')) or len(t) > 100):
        continue
      else:
        out += [t]   
  return out


response =  requests.get(url)
texts = [line.decode('latin-1') for line in response.iter_lines()]
print(f"No. of lines: {len(texts)}")
clean_texts = fix_texts(texts)
(data_dir/'text_clean.txt').open(mode='wt').writelines(f"{line}\n" for line in clean_texts)

No. of lines: 18035


## Define Tokenizer (which uses SentencePiece), Corpus ( data handler)

In [0]:
class Tokenizer():
    
    ID_UNK = 0
    ID_SOS = 1
    ID_EOS = 2
    
    def __init__(self, model_path:str):
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str(model_path))
        self.id_unk = self.sp.unk_id()
        self.tk_unk = self.sp.IdToPiece(self.id_unk)
        self.tk_sos = self.sp.IdToPiece(self.ID_SOS)
        self.tk_eos = self.sp.IdToPiece(self.ID_EOS)
        self.vocab_size = len(self.sp)
        logger.info(f"Initialized SentPieceProcessor from {model_path}")
    
    def numericalize(self, tokens: List[str]) -> List[List[int]]:
        if isinstance(tokens, str): tokens = [tokens]
        ids =  [self.sp.EncodeAsIds(s) for s in tokens]
        if len(ids) == 1: ids=ids[0]
        return ids

    def piecify(self, tokens: List[str]) -> List[List[str]]:
        if isinstance(tokens, str): tokens = [tokens]
        return [self.sp.EncodeAsPieces(s) for s in tokens]
    
    def textify(self, ids: List[int]) -> str:
        if isinstance(ids, list) and isinstance(ids[0], np.generic): 
            ids = [int(x) for x in ids]
        if not isinstance(ids, list) and not isinstance(ids[0], int):
            raise TypeError("Argument `ids` has to be a list of integers.")            
        return self.sp.DecodeIds(ids)
    
    @classmethod
    def from_file(cls, input_file:str, output_path:str='default', vocab_size:int=16000, 
                  char_cov:float=1.0, model_type:str='unigram'):
        
        assert model_type in ['unigram', 'bpe', 'char', 'word']
        assert 0 < char_cov <= 1
        input_file = str(input_file)
        output_file =  os.path.splitext(str(output_path))[0]
        ext = '.model'
        train_cmd = f"--input={input_file} --model_prefix={output_file}"\
                    f" --vocab_size={vocab_size} --character_coverage={char_cov} --model_type={model_type}"

        logger.info(f"Train command: {train_cmd}")
        logger.info(f"Started training SentencePiece model...")
        ret = spm.SentencePieceTrainer.Train(train_cmd)
        logger.info(f"Exit code: {int(ret)}")
        return cls(output_file+ext)
      
def batchify(data, bsz):
    # work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

class Corpus(object):

  def __init__(self, file_path, tokenizer, bs=20):
       
        self.processor = tokenizer
        self.id_sos, self.id_eos = self.processor.ID_SOS, self.processor.ID_EOS
        self.bs = bs
        self.data = self.tokenize(file_path)        
        self.vocab_size = self.processor.vocab_size
        
  def tokenize(self, path):
      """Tokenizes a text file."""
      ids = []
      with open(path, 'r') as f:
          for line in f:
              numericalized = self.processor.numericalize(line)
              ids.extend([self.id_sos] + numericalized + [self.id_eos])

      return batchify(torch.LongTensor(ids), self.bs)

## Define RNN model architecture,  training loop and helpers

In [0]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, emsize, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, emsize)
        assert (rnn_type in ['LSTM', 'GRU']), "Arg `rnn_type` has to be one of {GRU, LSTM}."
        self.rnn = getattr(nn, rnn_type)(emsize, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        if tie_weights:
            if nhid != emsize:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)
       
      
def repackage_hidden(h):
  """Wraps hidden states in new Tensors, to detach them from their history."""
  if isinstance(h, torch.Tensor):
    return h.detach()
  else:
    return tuple(repackage_hidden(v) for v in h)
      
def get_num_params(model):
  return sum([p.nelement() for p in model.parameters() if p.requires_grad])

def get_batch(source, i, bptt):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
  
def train_epoch(train_data, model, vocab_size, bs=16, bptt=20, clip=.25):
    model.train()
    total_loss = 0.0
    hidden = model.init_hidden(bs)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i, bptt)

        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        total_loss += len(data) * loss.item()
        
    return total_loss / (len(train_data) - 1)

In [100]:
# INIT AND TRAIN TOKENIZER FROM CLEAN TEXTS
bs = 32

model_type = 'unigram' # can be "bpe"/"unigram" to support wordpieces

vocab_size = 16000 # how many wordpieces to consider

tokenizer = Tokenizer.from_file(data_dir/'text_clean.txt', 
                                output_path=data_dir/'tokenizer', char_cov=1.0,
                                model_type=model_type, vocab_size=vocab_size) 

# INIT CORPUS FROM CLEAN TEXTS AND TOKENIZER WITH BATCH_SIZE `BS`

corpus = Corpus(data_dir/'text_clean.txt', tokenizer=tokenizer, bs=bs)

2019-04-10 15:46:56,868 : INFO : Train command: --input=/content/data/jozsef-wp/text_clean.txt --model_prefix=/content/data/jozsef-wp/tokenizer --vocab_size=16000 --character_coverage=1.0 --model_type=unigram
2019-04-10 15:46:56,871 : INFO : Started training SentencePiece model...
2019-04-10 15:46:57,931 : INFO : Exit code: 1
2019-04-10 15:46:57,962 : INFO : Initialized SentPieceProcessor from /content/data/jozsef-wp/tokenizer.model


In [101]:
# LSTM PARAMETERS
model_type='GRU'

emsize = 300
nhid = 300

nlayers = 1

dropout = 0.25
clip = 2.5

tied = True

bptt = 80

lr = 1e-3

# INIT LSTM MODEL, LOSS FUNCTION AND OPTIMIZER

model = RNNModel(model_type, corpus.vocab_size, emsize, nhid, nlayers, dropout, tied).to(device)
print(f"No. of parameters: {get_num_params(model)}")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

No. of parameters: 5357800


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


## Let's train the model

In [102]:
epochs = 150

start_time = time.time()
try:
  
  for epoch in range(1, epochs+1):

    loss = train_epoch(corpus.data, model, corpus.vocab_size, 
                       clip=clip, bs=bs, bptt=bptt)
    elapsed = time.time() - start_time
    print("epoch {:3d} | loss {:5.2f} | perplexity {:8.2f}| elapsed {:5.2f}s ".format(epoch, loss, math.exp(loss), elapsed))

except KeyboardInterrupt:
  print('-' * 89)
  print('Exiting from training early')

finally:
  params = {"model_type": model_type,
            "ntoken": corpus.vocab_size,
            "emsize": emsize,
            "nhid": nhid,
            "nlayers": nlayers,
            "dropout": dropout,
            "tied": tied}
 
  with open(data_dir/'model_state.pth', 'wb') as f:
    torch.save({"state_dict": model.state_dict(),
                "params": params}, f)
    
zipfile = zip_dir(data_dir, data_dir.name)

epoch   1 | loss  7.10 | perplexity  1212.40| elapsed  4.34s 
epoch   2 | loss  6.04 | perplexity   419.14| elapsed  8.51s 
epoch   3 | loss  5.84 | perplexity   344.08| elapsed 12.79s 
epoch   4 | loss  5.71 | perplexity   300.47| elapsed 16.97s 
epoch   5 | loss  5.58 | perplexity   264.88| elapsed 21.25s 
epoch   6 | loss  5.46 | perplexity   235.19| elapsed 25.46s 
epoch   7 | loss  5.34 | perplexity   209.37| elapsed 29.79s 
epoch   8 | loss  5.23 | perplexity   187.67| elapsed 34.00s 
epoch   9 | loss  5.13 | perplexity   168.70| elapsed 38.18s 
epoch  10 | loss  5.02 | perplexity   151.64| elapsed 42.45s 
epoch  11 | loss  4.91 | perplexity   135.89| elapsed 46.75s 
epoch  12 | loss  4.81 | perplexity   122.17| elapsed 50.95s 
epoch  13 | loss  4.70 | perplexity   109.71| elapsed 55.14s 
epoch  14 | loss  4.59 | perplexity    98.35| elapsed 59.44s 
epoch  15 | loss  4.48 | perplexity    88.21| elapsed 63.63s 
epoch  16 | loss  4.37 | perplexity    79.10| elapsed 67.92s 
epoch  1

## Generate verses! 

In [0]:
from random import sample
import torch

def is_unbalanced(s):
  if s.count('"') % 2 != 0 or s.count('(') != s.count(')'):
    return True
  else:
    return False

def sample_punkt():
  return sample(['.', '?', '!'], 1)[0]

def parse_last_line(s):
  l = list(s)
  
  if l[-1] == ',': 
    l[-1] = sample_punkt()
  if l[-1] not in list('.?!'): 
    l.append(sample_punkt())  
  return "".join(l)
  
def generate_line(model, hidden=None, temp=1.0, 
               sos_id=1, eos_id=2, unk_id=0, max_len=None):
  """Generate line from `model` with `hidden` state at `temp`."""
  ids = []
  
  if hidden is None:
    hidden = model.init_hidden(1)
  
  input = torch.tensor([sos_id], dtype=torch.long).reshape(1,1).to(device)
  
  id = 0
  while id != eos_id and len(ids)<max_len :
    output, hidden = model(input, hidden)
    probs = output.squeeze().div(temp).exp().cpu() 
    id = torch.multinomial(probs, num_samples=1).item() 
    if id == sos_id or id == unk_id: continue
    input.fill_(id)
    ids += [id]
  
  return ids, hidden

def generate(model, tokenizer, num_lines=8, min_len=8, max_len=15,
             unk_id=0, sos_id=1, eos_id=2, temp=0.6):
  """ 
  Generate a verse consisting of `num_lines` lines of max. length `max_tokens`.
  Since the hidden state is passed onto the next line, 
  observing some cross-line consistency would be expected, or less
  optimistically, at least grammatically correct sentences.
  NOTE: line length can be tuned by changing max_tokens (i.e. subword pieces).
  
  Args:
    model: Trained PyTorch language model
    tokenizer: SentencePiece tokenizer
    temp: Temperature parameter; lower: more conservative, higher: more diverse
    num_lines: No. of lines to generate.
    max_len: Max no. of tokens per line (not words!)
    sos_id: Start of sequence id in vocabulary
    eos_id: End of sequence id in vocabulary
  
  Returns: list of strings
  """
  model.eval()
  
  lines = []
  line_cnt = 0
  hidden = model.init_hidden(1)

  with torch.no_grad():
    
    while line_cnt != num_lines:
      try:

        ids, hidden = generate_line(model, hidden=hidden, temp=temp, max_len=max_len,
                                    sos_id=sos_id, eos_id=eos_id, unk_id=unk_id)
        
        if len(ids) <= min_len: raise Exception
        line = tokenizer.textify(ids).strip()
        
        if line.startswith(tuple("-?!.,()")): raise Exception
        if is_unbalanced(line): raise Exception
        
        lines += [line]
        line_cnt +=1
        
      except Exception as e:
        pass
    
  last_line = lines.pop()
  l = parse_last_line(last_line)
  lines.append(l)
  
  return lines

In [109]:
generate(model, tokenizer, temp=0.5, num_lines=10, min_len=8, max_len=14)

['Űr a lelkem, a lét türelme.',
 's a kicból a halál!',
 'adás, a bérházon a tenger',
 'a sokszor be nem al elé.',
 'a kék kein belieber leser.',
 's a kis öreg hosszan bámul az eltünő csapat után',
 'meg erős, a tet nem fogom megcsókolni',
 'a vágy, a jó és az ágy,',
 'több a mi fehér kövér, ki ád',
 'az Isten is - a hold, a csatorna felé fodorul!']

In [108]:
generate(model, tokenizer, temp=0.6, num_lines=20, max_len=15)

['Űr a lelkem. A tömegeken így lesz neki',
 'kit rajta nem hallod, csak a kedves',
 'a hajnal, a szó a csontunk',
 'is látni magát a Szél, a Nap, a halál.',
 'S a mult, a szó, a jog,',
 'az asszony, amíg a munkás beleöltözött,',
 'amíg de te megérkezik a meztelen leányzó.',
 's a kék, az asszony is,',
 's az a kétszert szerez:',
 'a csönd, a halál nem rossz, én a hajat kedveltem mindig,',
 'a bagoly hd, hogy kösse letöröm.',
 's a kis öreg hosszan bámul az eltünő csapat után,',
 'S a szép vágyig, ha nem tud ott ketté a faág,',
 's hánynyolhonban a földtekén:',
 'a vágy, de mi a Nap, a világ!',
 'az asszony, a létra, a nap',
 's a nagy városokat, ahol a gyümölcse, te e a föld,',
 'az Isten is lesz, hogy több a pja,',
 'a fény, mert a kés éle,',
 'az a szép, hogy a fény meg!']

In [0]:
generate(model, tokenizer, temp=0.7, num_lines=10, max_len=15)

['a föld, az ég, a csönd',
 'a kő, a mi nem is,',
 'csak egy darabos kontyba kötjük, s egy fény az ég.',
 'a halak, mely ősz hull a fösvény országban, ahol',
 'a multé, fehér habok fölött.',
 'a mezei ház mellé, hol a kert bokrain túl',
 'kenyeret, hogy húzódik fagyott s a fény,',
 'a csont a csendjét a ház,',
 'a légy, az idegeket, izmokat,',
 'a kín, az idegem, hajnal harmat.']